Surface Fuel Updates.. buckle up

There will be 3 GridFire runs representing Year 5, Year 10, and Year 20.

We have 4 year intervals in our ee_treatment simulation images, the pixel values and their Year Interval correspond:


|Pixel Value | Year Interval|
|---|----|
|1 |years 1-5|
|2 |years 6-10|
|3 |years 11-15|
|4 |years 16-20|

We want to generate newly updated FM40 layers for each GridFire run year. 
Since there is a mismatch in the cadence of GridFire run year intervals and our ee_treatments() year intervals,
we will be updating only those treated pixels with value 1, 2, and 4 from our ee_treatment() images.

For each of those pixel values (1,2,4) we will create a new updated FM40 image. The workflow is this:
* mask to only pixels with that given value, 
* use a simplified lookup table .csv on Cloud Storage (containing from_codes and to_codes) to remap the baseline FM40 values to their updated values in those treated pixels
* then fill in the 'untreated pixels' with the baseline FM40 image

Expected Output is three Updated FM40 images per RunID

Example: RUNID1 = SN WUI 500k
Outputs:
* GEE Asset - projects/mas-gee/assets/treatment_scenarios/RunID1/FM40_GFrun1 (updated in ee_treatment pixels==1)
* GEE Asset - projects/mas-gee/assets/treatment_scenarios/RunID1/FM40_GFrun2 (updated in ee_treatment pixels==2)
* GEE Asset - projects/mas-gee/assets/treatment_scenarios/RunID1/FM40_GFrun3 (updated in ee_treatment pixels==4)

We also are exporting them to Google Drive in case there is a need
* Google Drive - RunDID1_FM40_GFrun1.tif
* Google Drive - RunDID1_FM40_GFrun2.tif
* Google Drive - RunDID1_FM40_GFrun3.tif


In [25]:
import ee
import os
import math
from datetime import datetime
from src.utils import ee_treatments
from src.utils import runs
from src.utils.check_exists import check_exists
from src.utils.yml_params import get_export_params
from src.utils.ee_csv_parser import parse_txt, to_numeric
yml_file = os.path.join(os.getcwd(),'config.yml')
CRS,EXPORT_SCALE = get_export_params(yml_file)

ee.Initialize(project='mas-gee')
%load_ext autoreload
%autoreload 2

# In development phase
tag = 'fm40_updates'
ee.data.setWorkloadTag(tag)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


INPUT YOUR TREATED IMAGE PATH BELOW, THEN RUN ALL

In [26]:
# Plug in treatment scenario ee.Image path
ee_treated_path = "projects/mas-gee/assets/treatment_scenarios/RunID36/CC_Fire_2m_20231018"
retreatment_path = "projects/mas-gee/assets/treatment_scenarios/RunID36/CC_Fire_2m_20231103_retreatment_only"

In [27]:
ee_treated = ee.Image(ee_treated_path)
retreated = ee.Image(retreatment_path)
baseline_fm40 = ee.Image("projects/pyregence-ee/assets/conus/landfire/fbfm40/LF2022_FBFM40_220")

In [28]:
# test ee blob table encoding working properly
blob = ee.Blob("gs://landfire/LFTFCT_tables/million-acres-LUTs/LANDFIRE fuel model subs - FM40.csv")
table = parse_txt(blob)

# read in the encoded value list as numeric
from_codes = to_numeric(ee.List(table.get("Original FM40_number")))
# read the list of values to remap to as numeric
to_codes = to_numeric(ee.List(table.get("Updated FM40_number")))

print(from_codes.getInfo())
print(to_codes.getInfo())

[91, 92, 93, 98, 99, 101, 102, 103, 104, 121, 122, 123, 141, 142, 143, 144, 145, 146, 147, 161, 162, 163, 165, 181, 182, 183, 184, 185, 186, 187, 188, 189, 201, 202]
[91, 92, 93, 98, 99, 101, 101, 101, 101, 121, 121, 121, 141, 141, 141, 141, 142, 141, 142, 161, 161, 161, 161, 181, 181, 181, 181, 181, 181, 181, 181, 181, 201, 201]


<!-- For 2.3M intensity, SC and CC regions, we have re-treatments in road grid pixels, and unique floating point numbers distinguish a pixel that was treated twice from pixels only treated once. (See the `SCandCC_*_2Mcustom.ipynb` notebooks)

For run1 (Year5): 
* crosswalk only pixel value 1 (areas only treated once, no re-treatment)

For run2 (Year10):
* crosswalk pixel value 1.5 (areas treated in YI1 (Year5) and re-treated in YI2 (Year10))
* crosswalk pixel value 2 (areas only treated in YI2 (Year10))

For run3 (Year20):
* crosswalk pixel value 3.5 (areas treated in YI3 (Year15) and re-treated in YI4 (Year20))
* crosswalk pixel value 4 (areas only treated in YI4 (Year20)) -->

## 2.M SC/CC Custom Workflow
In `TreatmentSimulator_2MCustom` notebooks we exported `ee_treatments` and `retreatment_only` images separately

In this Notebook we:
1) FM40 crosswalk each treatment image separately for the three GF runs 

*run1 (Year5 | pixel value=1), run2 (Year10 | pixel value=2) and run3 (Year20 | pixel value=4)*


2) then merging the two updated FM40s together in each GFrun pair, taking the min in overlapping pixels (remaps only go down, see above `from_codes` and `to_codes` lists)

In [29]:
# dumb mode - it works but i'm ashamed

# run 1
if 'WUI' in ee_treated_path: # when we ran ee_treatmetns for WUI, 1's and 4's shared same footprint and 4 burned over 1's (took the max)
    run1_ee_treatment = ee_treated.eq(4).selfMask()
else:
    run1_ee_treatment = ee_treated.eq(1).selfMask() # for RFFC and Fire runs

run1_ee_treatment_remap = baseline_fm40.updateMask(run1_ee_treatment).remap(from_codes,to_codes)
run1_ee_treatment_fm40 = run1_ee_treatment_remap.unmask(baseline_fm40).rename('new_fm40').set('GFyearInterval',1)

# we don't re-treat in Year5 (run1 | YI1 whatever you want to call it) 
run1_fm40_mosaic = run1_ee_treatment_fm40 

# run 2
run2_ee_treatment = ee_treated.eq(2).selfMask()
run2_retreatment = retreated.eq(2).selfMask()

run2_ee_treatment_remap = baseline_fm40.updateMask(run2_ee_treatment).remap(from_codes,to_codes)
run2_ee_treatment_fm40 = run2_ee_treatment_remap.unmask(baseline_fm40).rename('new_fm40').set('GFyearInterval',2)

run2_retreatment_remap = baseline_fm40.updateMask(run2_retreatment).remap(from_codes,to_codes)
run2_retreatment_fm40 = run2_retreatment_remap.unmask(baseline_fm40).rename('new_fm40').set('GFyearInterval',2)

run2_fm40_mosaic = (ee.Image.cat(run2_ee_treatment_fm40,
                                 run2_retreatment_fm40)
                    .reduce(ee.Reducer.min())
                    .rename('new_fm40')
                    .set('GFyearInterval',2))

# run 3  
run3_ee_treatment = ee_treated.eq(4).selfMask()
run3_retreatment = retreated.eq(4).selfMask()

run3_ee_treatment_remap = baseline_fm40.updateMask(run3_ee_treatment).remap(from_codes,to_codes)
run3_ee_treatment_fm40 = run3_ee_treatment_remap.unmask(baseline_fm40).rename('new_fm40').set('GFyearInterval',2)

run3_retreatment_remap = baseline_fm40.updateMask(run3_retreatment).remap(from_codes,to_codes)
run3_retreatment_fm40 = run3_retreatment_remap.unmask(baseline_fm40).rename('new_fm40').set('GFyearInterval',2)

run3_fm40_mosaic = (ee.Image.cat(run3_ee_treatment_fm40,
                                 run3_retreatment_fm40)
                    .reduce(ee.Reducer.min())
                    .rename('new_fm40')
                    .set('GFyearInterval',3))

updated_fm40s = [run1_fm40_mosaic,
                 run2_fm40_mosaic,
                 run3_fm40_mosaic]

In [30]:
import geemap
Map = geemap.Map()

Map.addLayer(baseline_fm40,{'min':90,'max':205},'baseline fm40')

# run 1 output steps
Map.addLayer(run1_ee_treatment,{'min':1,'max':4,'palette':['blue','red','green','purple']},' run1 ee_treatments')
Map.addLayer(run1_fm40_mosaic,{'min':90,'max':205,},'updated fm40 mosaic run1') 

# run 2 output steps
Map.addLayer(run2_ee_treatment,{'min':1,'max':4,'palette':['blue','red','green','purple']},'run2 ee_treatments')
Map.addLayer(run2_retreatment,{'min':1,'max':4,'palette':['blue','red','green','purple']},'run2 retreated')
Map.addLayer(run2_ee_treatment_fm40,{'min':90,'max':205,},'run2 ee_treatment fm40')
Map.addLayer(run2_retreatment_fm40,{'min':90,'max':205,},'run2 retreatment fm40') 
Map.addLayer(run2_fm40_mosaic,{'min':90,'max':205,},'run2 updated fm40 mosaic') 

# run 3 output steps
Map.addLayer(run3_ee_treatment,{'min':1,'max':4,'palette':['blue','red','green','purple']},'run3 ee_treatments')
Map.addLayer(run3_retreatment,{'min':1,'max':4,'palette':['blue','red','green','purple']},'run3 retreated')
Map.addLayer(run3_ee_treatment_fm40,{'min':90,'max':205,},'run3 ee_treatment fm40')
Map.addLayer(run3_retreatment_fm40,{'min':90,'max':205,},'run3 retreatment fm40') 
Map.addLayer(run3_fm40_mosaic,{'min':90,'max':205,},'run3 updated fm40 mosaic') 

# Map.centerObject(treated,11)

Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

Export each updated FM40 (run1,run2,run3)

In [31]:
# Export each ee_treatment FM40 in list
# use California as output region
ca = ee.FeatureCollection("TIGER/2018/States").filter(ee.Filter.eq("NAME","California")).first().geometry()

# use RUNID string as sub-folder in output Gdrive folder
runID_drive = os.path.basename(os.path.dirname(ee_treated_path)) # to get RUNID string

for img in list(range(3)):
    output = ee.Image(updated_fm40s[img]).toByte()
    year_interval = output.get('GFyearInterval').getInfo()
    desc = f"UpdateFM40_GFrun{year_interval}"
    output_path = os.path.dirname(ee_treated_path)+"/"+f"FM40_GFrun{year_interval}"
    
    ee_treatments.export_img(output,desc,output_path,ca,30,'EPSG:3310')
    
    ee_treatments.export_image_to_drive(image=output,
                                        description=desc+'-Drive',
                                        region=ca,
                                        scale=30,
                                        crs='EPSG:3310',
                                        folder=f"MAS_FM40_for_GridFire",
                                        fileNamePrefix=f"{runID_drive}_FM40_GFrun{year_interval}",
                                        maxPixels=1e12)

Export Started for projects/mas-gee/assets/treatment_scenarios/RunID36/FM40_GFrun1
Export Started (Drive): RunID36_FM40_GFrun1
Export Started for projects/mas-gee/assets/treatment_scenarios/RunID36/FM40_GFrun2
Export Started (Drive): RunID36_FM40_GFrun2
Export Started for projects/mas-gee/assets/treatment_scenarios/RunID36/FM40_GFrun3
Export Started (Drive): RunID36_FM40_GFrun3


Couldn't figure out the bug with this more 'elegant' function-based code so i went with barbarian dumb mode you see above

In [32]:
# def crosswalk_ee_trt(trt_img):

#     year5_treated = trt_img.eq(1).selfMask()
#     year5_remap = baseline_fm40.updateMask(year5_treated).remap(from_codes,to_codes) # remap FM40 in specific treated areas
#     year5_output = year5_remap.unmask(baseline_fm40).rename('new_fm40').set('GFyearInterval',1)

#     year10_treated = trt_img.eq(2).selfMask()
#     year10_remap = baseline_fm40.updateMask(year10_treated).remap(from_codes,to_codes) # remap FM40 in specific treated areas
#     year10_output = year10_remap.unmask(baseline_fm40).rename('new_fm40').set('GFyearInterval',2)
    
#     year20_treated = trt_img.eq(4).selfMask()
#     year20_remap = baseline_fm40.updateMask(year20_treated).remap(from_codes,to_codes) # remap FM40 in specific treated areas
#     year20_output = year20_remap.unmask(baseline_fm40).rename('new_fm40').set('GFyearInterval',3)

#     updated_fm40s = [year5_output,
#                             year10_output,
#                             year20_output]
#     return updated_fm40s

# def crosswalk_retreatments(trt_img):
#     # we don't retreat in Year5 so we just use the baseline fm40
#     year5_output = baseline_fm40.rename('new_fm40').set('GFyearInterval',1)

#     year10_treated = trt_img.eq(2).selfMask()
#     year10_remap = baseline_fm40.updateMask(year10_treated).remap(from_codes,to_codes) # remap FM40 in specific treated areas
#     year10_output = year10_remap.unmask(baseline_fm40).rename('new_fm40').set('GFyearInterval',2)

#     year20_treated = trt_img.eq(4).selfMask()
#     year20_remap = baseline_fm40.updateMask(year20_treated).remap(from_codes,to_codes) # remap FM40 in specific treated areas
#     year20_output = year20_remap.unmask(baseline_fm40).rename('new_fm40').set('GFyearInterval',3)

#     updated_fm40s = [year5_output,
#                             year10_output,
#                             year20_output]
#     return updated_fm40s

# ee_treatment_fm40s = crosswalk_ee_trt(ee_treated)
# retreatment_fm40s = crosswalk_retreatments(retreated)

# fm40_mosaics = []
# for i in list(range(len(ee_treatment_fm40s))):
#     ee_treatment_fm40s_i = ee_treatment_fm40s[i]
#     retreatment_fm40s_i = retreatment_fm40s[i]
#     print(ee_treatment_fm40s_i.bandNames().getInfo())
#     print(retreatment_fm40s_i.bandNames().getInfo())

#     fm40_pair = [ee_treatment_fm40s[i],retreatment_fm40s[i]]
#     # [print(f.get('GFyearInterval').getInfo()) for f in fm40_pair]
#     fm40_mosaic = (ee.Image.cat(fm40_pair)
#                    .reduce(ee.Reducer.min())
#                    .toInt8()
#                    .rename('new_fm40')
#                    .set('GFyearInterval',i+1))
#     fm40_mosaics.append(fm40_mosaic)
#     break

# print(ee_treatment_fm40s[0].getInfo())